In [1]:
import panel as pn
import altair as alt
import pandas as pd
from vega_datasets import data
import datetime as dt
from altair import datum

In [2]:
alt.renderers.enable('default')
pn.extension('vega')

In [3]:
source = data.stocks()
source = pd.DataFrame(source)
source.head()

,symbol,date,price
0,MSFT,2000-01-01,39.81
1,MSFT,2000-02-01,36.35
2,MSFT,2000-03-01,43.22
3,MSFT,2000-04-01,28.37
4,MSFT,2000-05-01,25.45


In [4]:
# set a title for your dashboard
title = '## Stock Price Dashboard'

subtitle = 'This dashboard allows you to select a company and date range to see stock prices.'

# create list of company names (tickers)
tickers = ['AAPL', 'GOOG', 'IBM', 'MSFT']

# this creates the dropdown widget
ticker = pn.widgets.Select(name='Company', options=tickers)

# this creates the date range slider
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2001, 1, 1), end=dt.datetime(2010, 1, 1),
    value=(dt.datetime(2001, 1, 1), dt.datetime(2010, 1, 1))
)

# tell Panel what your plot "depends" on. 
# This defines what should trigger a change in the chart.
# both values in depends() will be used in our below Altair chart as filters
@pn.depends(ticker.param.value, date_range_slider.param.value)
def get_plot(ticker, date_range): # start function

    # Load and format the data
    df = source # define df
    df['date'] = pd.to_datetime(df['date']) # format date as datetime
    
    # create a date filter that uses values from the date range slider
    start_date = date_range_slider.value[0] # store the first date range slider value in a var
    end_date = date_range_slider.value[1] # store the end date in a var
    mask = (df['date'] > start_date) & (df['date'] <= end_date) # create filter mask for the dataframe
    df = df.loc[mask] # filter the dataframe
    
    # create the Altair chart object
    chart = alt.Chart(df).mark_area(color="#0c1944", opacity=0.8).encode(x='date', y='price', tooltip=alt.Tooltip(['date','price'])).transform_filter(
        (datum.symbol == ticker) # this ties in the filter from the dropdown selection
    )
    
    return chart

# create the Panel object, passing in all smaller objects

dashboard = pn.Row(
    pn.Column(title, subtitle, ticker, date_range_slider),
    get_plot # draw chart function!
)

In [5]:
dashboard.servable()

Row
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Select(name='Company', options=['AAPL', 'GOOG', ...], value='AAPL')
        [3] DateRangeSlider(end=datetime.datetime(2010, ..., name='Date Range Slider', start=datetime.datetime(2001, ..., value=(datetime.datetime(2001, ..., value_end=datetime.datetime(2010, ..., value_start=datetime.datetime(2001, ...)
    [1] ParamFunction(function)